In [13]:
import openai
import json

model = "gpt-3.5-turbo-0613"

def get_current_weather(location, unit = "fahrenheit"):
    return "70"

functions = [
     {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    } 
  ]

available_functions = {
    "get_current_weather": get_current_weather,
}

In [16]:
def run_conversation(content):
  messages = [
    {"role": "user", "content": content},
    ]
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  messages.append(response_message)  # extend conversation with assistant's reply

  # Step 2: check if GPT wanted to call a function
  while response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Step 4: send the info on the function call and function response to GPT
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    response_message = response["choices"][0]["message"]
    messages.append(response_message)  # extend conversation with assistant's reply

  for message in messages:
    if message.get("content"):
      print(message["role"] + ": " + message["content"])
    if message.get("function_call"):
      print(" -> calling function " + message["function_call"]["name"] + " with arguments " + json.dumps(message["function_call"]["arguments"]))
  

In [17]:
run_conversation("What is the weather like in Boston?")

user: What is the weather like in Boston?
 -> calling function get_current_weather with arguments "{\n  \"location\": \"Boston, MA\"\n}"
function: 70
assistant: The current weather in Boston, MA is 70°F.
